In [1]:
import ITensors:op
import Pkg; Pkg.add("HDF5")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("PyFormattedStrings")
import Pkg; Pkg.add("LinearAlgebra")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using ITensors, ITensorMPS
import ITensors:op
using HDF5
using Plots
using PyFormattedStrings
using LinearAlgebra

In [3]:
include("overload_Spin=3h.jl")

In [4]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 4
N = 30

  # Make an array of N Index objects with alternating
  # "S=1/2" and "S=1" tags on odd versus even sites
  # (The first argument n->isodd(n) ... is an
  # on-the-fly function mapping integers to strings)
  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J = 1
  B = 0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    
nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

psi0 = random_mps(sites;linkdims=2)

energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(psi0,A)
ket1 = psi0[A]
for i=A+1:D
    ket1 *= psi0[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho)  #<-- this shows that the indices of rho corresponding to the site between B and C are the 3rd and 7th element of the inds(rho) vector.    
rho_ABC3 = rho      # delta(inds,inds) traces out the given inds (see the documentation)  
#rho_ABC3 = rho_ABC*delta(inds(rho)[4],inds(rho)[10])   
#rho_ABC3 = rho_ABC1*delta(inds(rho)[5],inds(rho)[12])  
#rho_ABC3 = rho_ABC2*delta(inds(rho)[6],inds(rho)[14]) 
#da, ua = eigen(rho_ABC1)


oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)

# Compute the eigenvalues of the partially transposed density matrix
da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L00=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
    
@show L00;
    
    
end   

After sweep 1 energy=-29.87909397552165  maxlinkdim=5 maxerr=5.01E-03 time=20.462
After sweep 2 energy=-30.61607382965059  maxlinkdim=40 maxerr=9.58E-12 time=0.344
After sweep 3 energy=-30.99078461965167  maxlinkdim=100 maxerr=5.83E-10 time=1.896
After sweep 4 energy=-31.006109571507942  maxlinkdim=150 maxerr=5.03E-10 time=7.941
After sweep 5 energy=-31.008009090953113  maxlinkdim=200 maxerr=2.38E-11 time=20.475
After sweep 6 energy=-31.008384581815747  maxlinkdim=201 maxerr=1.00E-11 time=19.015
After sweep 7 energy=-31.00845833188025  maxlinkdim=179 maxerr=9.99E-12 time=19.757
After sweep 8 energy=-31.008470275315215  maxlinkdim=142 maxerr=9.99E-12 time=16.891
After sweep 9 energy=-31.008471769936104  maxlinkdim=113 maxerr=9.99E-12 time=8.008
After sweep 10 energy=-31.008471979074564  maxlinkdim=96 maxerr=9.98E-12 time=7.620
After sweep 11 energy=-31.008472012371627  maxlinkdim=88 maxerr=9.97E-12 time=4.359
After sweep 12 energy=-31.008472014546577  maxlinkdim=80 maxerr=9.99E-12 time=

0.4716682759462508

In [5]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 5
N = 30

  # Make an array of N Index objects with alternating
  # "S=1/2" and "S=1" tags on odd versus even sites
  # (The first argument n->isodd(n) ... is an
  # on-the-fly function mapping integers to strings)
  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J=1
  B=0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    
nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

psi0 = random_mps(sites;linkdims=2)

energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(psi0,A)
ket1 = psi0[A]
for i=A+1:D
    ket1 *= psi0[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho)  #<-- this shows that the indices of rho corresponding to the site between B and C are the 3rd and 7th element of the inds(rho) vector.    
rho_ABC3 = rho * delta(inds(rho)[3],inds(rho)[8])     # delta(inds,inds) traces out the given inds (see the documentation)  
#rho_ABC3 = rho_ABC*delta(inds(rho)[4],inds(rho)[10])   
#rho_ABC3 = rho_ABC1*delta(inds(rho)[5],inds(rho)[12])  
#rho_ABC3 = rho_ABC2*delta(inds(rho)[6],inds(rho)[14]) 
#da, ua = eigen(rho_ABC1)


oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)

# Compute the eigenvalues of the partially transposed density matrix
da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L11=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L11;
    
    
end   

After sweep 1 energy=-30.407878469377472  maxlinkdim=5 maxerr=5.69E-03 time=0.088
After sweep 2 energy=-30.726267663155696  maxlinkdim=37 maxerr=9.42E-12 time=0.275
After sweep 3 energy=-30.99860790823104  maxlinkdim=100 maxerr=3.14E-10 time=1.895
After sweep 4 energy=-31.00751755175536  maxlinkdim=150 maxerr=9.14E-11 time=9.043
After sweep 5 energy=-31.008363697019675  maxlinkdim=191 maxerr=9.96E-12 time=15.230
After sweep 6 energy=-31.008462141954947  maxlinkdim=167 maxerr=1.00E-11 time=12.312
After sweep 7 energy=-31.00847114814014  maxlinkdim=135 maxerr=9.98E-12 time=7.481
After sweep 8 energy=-31.00847191298713  maxlinkdim=99 maxerr=9.95E-12 time=4.328
After sweep 9 energy=-31.008472001702344  maxlinkdim=88 maxerr=9.94E-12 time=3.521
After sweep 10 energy=-31.008472013832364  maxlinkdim=82 maxerr=9.96E-12 time=2.916
After sweep 11 energy=-31.008472014226516  maxlinkdim=76 maxerr=9.97E-12 time=3.052
After sweep 12 energy=-31.008472014262363  maxlinkdim=75 maxerr=1.00E-11 time=2.755

0.34451022636226364

In [6]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 6
N = 30

  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J=1
  B=0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    
    
nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

psi0 = random_mps(sites;linkdims=2)

energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(psi0,A)
ket1 = psi0[A]
for i=A+1:D
    ket1 *= psi0[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho) 
rho_ABC = rho * delta(inds(rho)[3],inds(rho)[9])    
rho_ABC3 = rho_ABC*delta(inds(rho)[4],inds(rho)[10])   



oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)


da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L22=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L22;
    
    
end   

After sweep 1 energy=-30.567744597360118  maxlinkdim=5 maxerr=3.64E-03 time=0.050
After sweep 2 energy=-30.770300996653734  maxlinkdim=36 maxerr=9.38E-12 time=0.309
After sweep 3 energy=-31.007464129634528  maxlinkdim=99 maxerr=9.95E-12 time=2.692
After sweep 4 energy=-31.008401014442477  maxlinkdim=140 maxerr=9.99E-12 time=7.069
After sweep 5 energy=-31.00846496983721  maxlinkdim=131 maxerr=1.00E-11 time=6.184
After sweep 6 energy=-31.00847131990674  maxlinkdim=115 maxerr=9.98E-12 time=8.206
After sweep 7 energy=-31.00847193620621  maxlinkdim=100 maxerr=9.98E-12 time=4.020
After sweep 8 energy=-31.00847200754967  maxlinkdim=95 maxerr=1.00E-11 time=6.944
After sweep 9 energy=-31.00847201400767  maxlinkdim=79 maxerr=9.99E-12 time=2.862
After sweep 10 energy=-31.00847201425  maxlinkdim=76 maxerr=9.99E-12 time=3.287
After sweep 11 energy=-31.00847201428992  maxlinkdim=75 maxerr=9.99E-12 time=2.394
After sweep 12 energy=-31.00847201433879  maxlinkdim=75 maxerr=9.93E-12 time=2.379
After swe

0.002907398055280686

In [7]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 7
N = 30

  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J=1
  B=0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)

nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

psi0 = random_mps(sites;linkdims=2)

energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(psi0,A)
ket1 = psi0[A]
for i=A+1:D
    ket1 *= psi0[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho) 
rho_ABC = rho * delta(inds(rho)[3],inds(rho)[10])    
rho_ABC1 = rho_ABC*delta(inds(rho)[4],inds(rho)[11])   
rho_ABC3 = rho_ABC1*delta(inds(rho)[5],inds(rho)[12]) 


oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)


da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L33=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L33;
    
    
end   

After sweep 1 energy=-30.24923821481245  maxlinkdim=5 maxerr=5.15E-03 time=0.039
After sweep 2 energy=-30.635669259450047  maxlinkdim=39 maxerr=9.20E-12 time=0.284
After sweep 3 energy=-30.955428192939983  maxlinkdim=100 maxerr=1.94E-10 time=2.009
After sweep 4 energy=-30.996932011681753  maxlinkdim=150 maxerr=5.25E-10 time=6.876
After sweep 5 energy=-31.006251060395652  maxlinkdim=200 maxerr=1.17E-10 time=25.200
After sweep 6 energy=-31.00807917737402  maxlinkdim=250 maxerr=1.22E-11 time=24.767
After sweep 7 energy=-31.008395772384144  maxlinkdim=229 maxerr=1.00E-11 time=21.001
After sweep 8 energy=-31.00845714723545  maxlinkdim=193 maxerr=9.95E-12 time=13.706
After sweep 9 energy=-31.0084691337457  maxlinkdim=154 maxerr=1.00E-11 time=8.507
After sweep 10 energy=-31.008471471387733  maxlinkdim=117 maxerr=9.99E-12 time=5.439
After sweep 11 energy=-31.00847191786593  maxlinkdim=102 maxerr=9.96E-12 time=3.565
After sweep 12 energy=-31.008472003282122  maxlinkdim=94 maxerr=9.98E-12 time=3

0.002758584113559288

In [8]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 8
N = 30

  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J=1
  B=0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)


nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

psi0 = random_mps(sites;linkdims=10)

energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(psi0,A)
ket1 = psi0[A]
for i=A+1:D
    ket1 *= psi0[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho) 
rho_ABC = rho * delta(inds(rho)[3],inds(rho)[11])    
rho_ABC1 = rho_ABC*delta(inds(rho)[4],inds(rho)[12])   
rho_ABC2 = rho_ABC1*delta(inds(rho)[5],inds(rho)[13]) 
rho_ABC3 = rho_ABC2*delta(inds(rho)[6],inds(rho)[14])

oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)


da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L44=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L44;
    
    
end   

After sweep 1 energy=-30.157610555253996  maxlinkdim=5 maxerr=5.31E-02 time=0.111
After sweep 2 energy=-30.62891258262704  maxlinkdim=39 maxerr=9.93E-12 time=0.271
After sweep 3 energy=-30.976516705685995  maxlinkdim=100 maxerr=6.44E-10 time=1.704
After sweep 4 energy=-31.00371350125463  maxlinkdim=150 maxerr=4.56E-10 time=7.697
After sweep 5 energy=-31.007699144486025  maxlinkdim=200 maxerr=3.36E-11 time=15.529
After sweep 6 energy=-31.00833527838955  maxlinkdim=194 maxerr=9.99E-12 time=14.951
After sweep 7 energy=-31.008448196555953  maxlinkdim=162 maxerr=1.00E-11 time=10.238
After sweep 8 energy=-31.00846806566183  maxlinkdim=132 maxerr=9.92E-12 time=7.118
After sweep 9 energy=-31.008471377162383  maxlinkdim=110 maxerr=9.94E-12 time=5.166
After sweep 10 energy=-31.008471907754558  maxlinkdim=92 maxerr=9.98E-12 time=4.083
After sweep 11 energy=-31.00847200351552  maxlinkdim=90 maxerr=9.99E-12 time=3.035
After sweep 12 energy=-31.008472013938352  maxlinkdim=82 maxerr=9.99E-12 time=2.7

0.0

In [ ]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 9
N = 30

  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J=1
  B=0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)


nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

psi0 = random_mps(sites;linkdims=10)

energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(psi0,A)
ket1 = psi0[A]
for i=A+1:D
    ket1 *= psi0[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho) 
rho_ABC = rho * delta(inds(rho)[3],inds(rho)[12])    
rho_ABC1 = rho_ABC*delta(inds(rho)[4],inds(rho)[13])   
rho_ABC2 = rho_ABC1*delta(inds(rho)[5],inds(rho)[14]) 
rho_ABC21 = rho_ABC2*delta(inds(rho)[6],inds(rho)[15])
rho_ABC3 = rho_ABC21*delta(inds(rho)[7],inds(rho)[16])

oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)


da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L55=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L55;
    
    
end   

In [ ]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 10
N = 30

  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J=1
  B=0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)


nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

psi0 = random_mps(sites;linkdims=10)

energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(psi0,A)
ket1 = psi0[A]
for i=A+1:D
    ket1 *= psi0[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho) 
rho_ABC = rho * delta(inds(rho)[3],inds(rho)[13])    
rho_ABC1 = rho_ABC*delta(inds(rho)[4],inds(rho)[14])   
rho_ABC2 = rho_ABC1*delta(inds(rho)[5],inds(rho)[15]) 
rho_ABC21 = rho_ABC2*delta(inds(rho)[6],inds(rho)[16])
rho_ABC22 = rho_ABC21*delta(inds(rho)[7],inds(rho)[17])
rho_ABC3 = rho_ABC22*delta(inds(rho)[8],inds(rho)[18])
    
oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)


da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L66=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L66;
    
    
end   

In [ ]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 11
N = 30

  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J=1
  B=0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)


nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

psi0 = random_mps(sites;linkdims=10)

energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(psi0,A)
ket1 = psi0[A]
for i=A+1:D
    ket1 *= psi0[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho) 
rho_ABC = rho * delta(inds(rho)[3],inds(rho)[14])    
rho_ABC1 = rho_ABC*delta(inds(rho)[4],inds(rho)[15])   
rho_ABC2 = rho_ABC1*delta(inds(rho)[5],inds(rho)[16]) 
rho_ABC21 = rho_ABC2*delta(inds(rho)[6],inds(rho)[17])
rho_ABC22 = rho_ABC21*delta(inds(rho)[7],inds(rho)[18])
rho_ABC23 = rho_ABC22*delta(inds(rho)[8],inds(rho)[19])
rho_ABC3 = rho_ABC22*delta(inds(rho)[9],inds(rho)[20])
    
    
oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)


da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L77=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L77;
    
    
end   

In [11]:
L00=0.4716682759462508; L11=0.34451022636226364; L22=0.002907398055280686; 
L33=0.002758584113559288; L44=0.0; L55= ; L66= ; L77=

In [12]:
using Plots
using LaTeXStrings

k=[0,1,2,3,4,5,6,7]
La=[L00,L11,L22,L33,L44,L55,L66,L77]

plot(k,La)